### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/News
/sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/MST


In [5]:
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder:
    df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/News/Data_News.xlsx
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/MST/Data_List_Company.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/MST/Data_Detail.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/MST/FDI_Industry_Code.xlsx


In [6]:
url = []
for i in list_file:
    Folder = i.split('/')[7]
    if Folder == 'News':
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/Project/Crawling Data/News/Data_News.xlsx']

In [7]:
def get_data(relative_url, file_url):
    header_BIHub = 'share_point_BIHub'
    config_BIHub = read_config_json(config_path, header_BIHub)
    BIHub = SharePoint(config_BIHub)

    sp_object = relative_url.split('/')[2].replace('-', '')
    data = BIHub.get_file(file_url)
    # Convert data to DataFrame if it's not already
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)

    # Remove irrelevant data
    data.dropna(axis=0, how='all', inplace=True)  # remove blank rows
    data.drop(data.columns[data.columns.str.contains('Unnamed')], axis=1, inplace=True)  # remove blank columns
    data.columns = data.columns.str.strip().str.replace(r'\s+', '', regex=True)  # strip column
    return data

## Import

In [8]:
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat = pd.DataFrame()
df_flat = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 
table_name = 'News'
columns_that_need_unidecode = ['Topic','News_Time', 'Title', 'Short_Description', 'Position']

for file_url in tqdm(url):
    data = get_data(relative_url, file_url)
    data = remove_unformated_character(data)
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    today = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    data['Import_Date']=today
    filtered_data = data[~data['Link'].isin(pd.read_sql_query("SELECT Link FROM DW.News", engine)['Link'])]
    get_db= pd.read_sql_query("SELECT * FROM DW.News", engine)
    df_get_db = pd.DataFrame(get_db)
    filtered_data = data[~data['Link'].isin(df_get_db['Link'])]
    filtered_data = filtered_data[~filtered_data['Title'].isin(df_get_db['Title'])]
    print(len(filtered_data))
    filtered_data.to_sql(table_name, engine, index=False, if_exists='append', schema='DW')

  0%|          | 0/1 [00:00<?, ?it/s]

305


100%|██████████| 1/1 [00:44<00:00, 44.77s/it]
